In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
procedureDF = pd.read_csv('procedure.csv')

In [2]:
persondrop=['location_id','provider_id','care_site_id','person_source_value','race_source_value',
            'gender_source_value','ethnicity_source_value','birth_datetime']
visitdrop=['visit_source_value','visit_start_date','visit_end_date','visit_start_datetime','visit_end_datetime']
proceduredrop=['procedure_source_value','procedure_source_concept_id','procedure_date']

In [3]:
procedureDF=procedureDF.drop(columns=persondrop)
procedureDF=procedureDF.drop(columns=visitdrop)
procedureDF=procedureDF.drop(columns=proceduredrop)

In [4]:
procedureDF.loc[pd.isnull(procedureDF['death_date']) , 'death'] = 0
procedureDF.loc[pd.isnull(procedureDF['death_date'])==False , 'death'] = 1

In [5]:
deathdrop=['death_date','death_datetime','death_type_concept_id','cause_source_concept_id']
procedureDF=procedureDF.drop(columns=deathdrop)

In [6]:
procedureDF['procedure_datetime']=pd.to_datetime(procedureDF['procedure_datetime'])

In [7]:
percent_missing = procedureDF.isnull().sum() * 100 / len(procedureDF)
missing_value_df = pd.DataFrame({'column_name': procedureDF.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
person_id,person_id,0.000000
visit_occurrence_id,visit_occurrence_id,0.000026
year_of_birth,year_of_birth,0.000000
ethnicity_concept_id,ethnicity_concept_id,0.000000
month_of_birth,month_of_birth,0.000000
day_of_birth,day_of_birth,0.000000
race_concept_id,race_concept_id,14.449761
gender_concept_id,gender_concept_id,0.000000
visit_concept_id,visit_concept_id,0.000026
visit_type_concept_id,visit_type_concept_id,0.000026


In [8]:
categorical= ['person_id','visit_occurrence_id','ethnicity_concept_id','race_concept_id','gender_concept_id',
             'visit_concept_id','visit_type_concept_id','procedure_concept_id','procedure_occurrence_id']
for col in categorical:
    procedureDF[col] = procedureDF[col].astype('object')
procedureDF['death'] = procedureDF['death'].astype(int)

In [9]:
procedureDF.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

person_id                    97918
visit_occurrence_id         943947
ethnicity_concept_id             2
race_concept_id                  6
gender_concept_id                2
visit_concept_id                 3
visit_type_concept_id            1
procedure_concept_id          6943
procedure_occurrence_id    4695626
dtype: int64

In [10]:
timeforalive=pd.to_datetime(procedureDF[procedureDF['death']==0].procedure_datetime.dropna().astype(np.int64).mean())
timefordeath=pd.to_datetime(procedureDF[procedureDF['death']==1].procedure_datetime.dropna().astype(np.int64).mean())
print(timeforalive)
print(timefordeath)

2014-05-15 08:44:57.258833920
2014-05-17 06:28:05.457545984


In [11]:
from scipy.stats import chi2_contingency

def chisq_of_df_cols(df, c1, c2):
    groupsizes = df.groupby([c1, c2]).size()
    ctsum = groupsizes.unstack(c1)
    # fillna(0) is necessary to remove any NAs which will cause exceptions
    return(chi2_contingency(ctsum.fillna(0)))

In [12]:
chisq_of_df_cols(procedureDF, 'procedure_concept_id', 'death')

(6995.1070085136125,
 0.32447645217002186,
 6942,
 array([[2.12412449e+05, 9.89612701e-01, 9.89612701e-01, ...,
         8.11482415e+01, 3.95845081e+01, 5.73975367e+01],
        [2.22955054e+03, 1.03872986e-02, 1.03872986e-02, ...,
         8.51758483e-01, 4.15491943e-01, 6.02463317e-01]]))